In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import Dataset
from PIL import Image
import matplotlib.pyplot as plt

IMAGE_FILE = "../output/std_images.npy"
VELOCITY_FILE = "../output/Town06_500s-3_velocity_Smaller.npy"
MODEL_PATH = "../Models/velocity_regressor.pth"

class VelocityDataset(Dataset):
    def __init__(self, image_path, velocity_path, transform=None):
        self.images = np.load(image_path)
        self.velocities = np.load(velocity_path)

        if self.images.ndim == 4 and self.images.shape[1] == 3:
            self.images = np.transpose(self.images, (0, 2, 3, 1)) 

        if self.images.max() <= 1.0:
            self.images = (self.images * 255).astype(np.uint8)

        self.transform = transform

    def __len__(self):
        return len(self.velocities)

    def __getitem__(self, idx):
        img = Image.fromarray(self.images[idx])
        velocity = self.velocities[idx]

        if self.transform:
            img = self.transform(img)

        return img, velocity

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

class VelocityRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        base = models.resnet18(pretrained=False)
        base.fc = nn.Linear(base.fc.in_features, 1)
        self.model = base

    def forward(self, x):
        return self.model(x)

#Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VelocityRegressor().to(device)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

#Load Data
dataset = VelocityDataset(IMAGE_FILE, VELOCITY_FILE, transform)

# how many samples to show
NUM_SAMPLES = 10  
indices = np.random.choice(len(dataset), NUM_SAMPLES, replace=False)

for i in indices:
    img_tensor, true_velocity = dataset[i]
    img_input = img_tensor.unsqueeze(0).to(device)

    with torch.no_grad():
        pred_velocity = model(img_input).item()

    img_disp = img_tensor.permute(1, 2, 0).cpu().numpy()  
    img_disp = (img_disp * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406]))
    img_disp = np.clip(img_disp, 0, 1)

    plt.imshow(img_disp)
    plt.title(f"Predicted: {pred_velocity:.2f} m/s | Actual: {true_velocity:.2f} m/s")
    plt.axis('off')
    plt.show()
